In [1]:
%load_ext autoreload
%autoreload 2

%pylab inline

import dataloaders
import models
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics 
import sklearn.cluster as cluster
import numpy as np 
import random
import classifiers
from torch.autograd import Variable
import torch
import torch.nn as nn 
import torch.optim as optim
import sklearn.model_selection as model_selection
import util
import json

from imblearn.under_sampling import RandomUnderSampler

Populating the interactive namespace from numpy and matplotlib


In [2]:
#input_file = '../data/soc-sign-Slashdot081106-cleaned.csv'
input_file = '../data/soc-sign-epinions-cleaned.csv'
delimiter = ','
ratio = 0.8
data = dataloaders.UnsplitDataset(filepath=input_file, ratio=ratio, delimiter=delimiter)

In [3]:
X, y = data.get_shuffled_data()

In [4]:
# parameters for PsuedoKernel 
num_nodes = data.get_num_nodes()
dims = 16
epochs = 100
lr = 0.15
lr_decay=0.0
weight_decay=0.0
lam = 0.00055
p = 2

In [5]:
model_fitter = models.fit_pseudo_kernel_model

In [6]:
num_splits = 5

In [7]:
kf = model_selection.KFold(n_splits=num_splits)

In [8]:
kf.get_n_splits(X)

5

In [25]:
len(X)/num_nodes
print(num_nodes)
print(len(X) * 0.8 * 0.148)


131828
99618.44480000001


In [17]:
len(y[y == 0]) * 0.8

98964.0

In [26]:
delta = 1
delta0 = 0.5
dims_array = [dims, 20, 20]
frac1 = 0.148
frac0 = 0.148
p0 = True if frac0 > 0 else 0

In [36]:
reports_pksem = []
reports_sine = []
operation = 'hadamard'
count = 1
for train_idx, test_idx in kf.split(X):
    
    print("======================================================================================")
    
    print("========================          Fold #{}                    ========================".format(count))
    
    
    print("======================================================================================")
    
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    
    # train and evaluate pksem
    kernel_model = models.fit_pseudo_kernel_model(num_nodes, dims, X, y, epochs=epochs, p=p, 
                                              lr=lr,lr_decay=lr_decay, lam=lam, 
                            weight_decay=weight_decay, undersample=True)
    clf_pksem = linear_model.LogisticRegression()
    report = classifiers.train_and_evaluate_classifier(clf_pksem, kernel_model, X_train, y_train, X_test, y_test,
                                                    operation=operation, undersample=True, ratio=2)
    reports_pksem.append(report)
    
    # train and evaluate SiNE
    triples, triples0 = util.triples_from_array(X_train, y_train)
    batch_size = int(frac1 * len(triples))
    batch_size0 = int(frac0 * len(triples0))
    sine_model = models.fit_sine_model(num_nodes, dims_array, triples, triples0, delta, delta0,
                                   batch_size, batch_size0, epochs, lr=lr, lr_decay=lr_decay,
                                  lam=lam, p=p, p0=p0)
    clf_sine = linear_model.LogisticRegression()
    report = classifiers.train_and_evaluate_classifier(clf_sine, sine_model, X_train, y_train, X_test, y_test,
                                                    operation=operation, undersample=True, ratio=2)
    reports_sine.append(report)
    count += 1
    
    
    
    
    

========================          Fold #1                    ========================
The loss at epoch  1  was  0.6993435025215149
The loss at epoch  2  was  0.8051214218139648
The loss at epoch  3  was  0.6685885787010193
The loss at epoch  4  was  0.6387786865234375
The loss at epoch  5  was  0.6212530136108398
The loss at epoch  6  was  0.6035315990447998
The loss at epoch  7  was  0.5711809396743774
The loss at epoch  8  was  0.5422701835632324
The loss at epoch  9  was  0.5254396796226501
The loss at epoch  10  was  0.5128694176673889
The loss at epoch  11  was  0.5055877566337585
The loss at epoch  12  was  0.5006917715072632
The loss at epoch  13  was  0.4950365424156189
The loss at epoch  14  was  0.48995521664619446
The loss at epoch  15  was  0.49123409390449524
The loss at epoch  16  was  0.4892246127128601
The loss at epoch  17  was  0.49451833963394165
The loss at epoch  18  was  0.4823787808418274
The loss at epoch  19  was  0.47689375281333923
The loss at epoch  20  was

KeyboardInterrupt: 

In [ ]:
def average_scores(reports, scores=['auc', 'average_percision_score', 'macro_f1', 'micro_f1', 
                                   'kappa', 'mathew']):
    score_lst = dict()
    for key in scores:
        score_lst[key] = []
    for report in reports:
        for key in scores:
            score_lst[key].append(report[key])
    
    score_avg = dict()
    score_std = dict()
    for key in scores:
        score_avg[key] = np.mean(score_lst[key])
        score_std[key] = np.std(score_lst[key])
        
    return score_avg, score_std
    
    

In [ ]:
r1 = average_scores(reports_pksem)

In [ ]:
r2 = average_scores(reports_sine)

In [ ]:
print(r1)

In [ ]:
print(r2)

In [ ]:
result_link_pred_file = 'link-pred?epinions.txt'

In [ ]:
with open(result_link_pred_file, 'a') as fp:
    line = '{0},{1},{2},{3},{4},{5}\n\n'.format(input_file, operation, r1[0], r1[1], r2[0], r2[1])
    fp.write(line)

In [ ]:
abs(0.923 - 0.930)/0.923 * 100